
# CortexRAG: End to End RAG with 3 lines of code

The `SnowVectorDB` and `SnowRAG` helpers were designed to simplify the developer experience when building simple RAG pipelines in Snowflake.

`SnowVectorDB` gives developers the ability to generate a Snowflake embeddings table directly from a raw data source in 2 lines of code. Whether your raw files are in a local directory or they're already loaded in a Snowflake Stage, this gives you the ability to automatically process the raw data files, generate their embeddings, create a new Snowflake embeddings table, and load the embeddings into the table as Snowflake Vector Data Types. 

`SnowRAG` allows developers to define a simple RAG pipeline in one line of code. Select the Snowflake embeddings table and the Snowflake Cortex model, and this module takes care of the rest. Under the hood SnowRAG uses Chain of Thought for response generation, but users have the flexbility to override that with custom prompts. 

Below we walk you through how to use `SnowVectorDB` in both operating modes - whether your data is stored in a local directory or in a Snowflake Stage. Then we show how simple it is to define a basic pipeline with `SnowRAG`.

#### Standard Snowflake Session Initialization

In [1]:
from snowvecdb import SnowVectorDB
from snowrag import SnowRAG
from snowflake.snowpark import Session
import os
from dotenv import load_dotenv
load_dotenv()

connection_parameters = {
    
    "account": os.getenv('SNOWFLAKE_ACCOUNT'),
    "user": os.getenv('SNOWFLAKE_USER'),
    "password": os.getenv('SNOWFLAKE_PASSWORD'),
    "role": os.getenv('SNOWFLAKE_ROLE'),
    "warehouse": os.getenv('SNOWFLAKE_WAREHOUSE'),
    "database": os.getenv('SNOWFLAKE_DATABASE'),
    "schema": os.getenv('SNOWFLAKE_SCHEMA')}  

session = Session.builder.configs(connection_parameters).create()

# Embeddings Generation

### Create Embeddings Table from local directory

Update the path to point to your relevant directory

In [3]:
SVDB = SnowVectorDB(snowflake_session=session) 
SVDB("ANNUAL_REPORT_EMBEDDINGS",data_source_directory="YOUR_LOCAL_DATA SOURCE_DIRECTORY/sample_documents")

Loading source data...
...data load complete.
Creating embeddings table...
Embeddings succesfully loaded in Snowflake table:ANNUAL_REPORT_EMBEDDINGS


### Create Embeddings Table from files in an existing stage

In [8]:
SVDB("ANNUAL_REPORT_EMBEDDINGS_STG",stage="YOUR_CUSTOM_STAGE")

Creating embeddings table...
Embeddings succesfully loaded in Snowflake table:ANNUAL_REPORT_EMBEDDINGS_STG


# Snowflake RAG

### Initialize RAG Chat Bot

In [4]:
rag = SnowRAG(embeddings_table="ANNUAL_REPORT_EMBEDDINGS",lm_model="mixtral-8x7b",snowflake_session=session)

### Run the program

In [7]:
rag("How many customers did Snowflake have in fiscal year 2024?")

[' 1. Snowflake had 9,437 total customers as of January 31, 2024.\n2. The fiscal year 2024 for Snowflake ended on January 31, 2024.\n3. The total customer count includes adjustments for acquisitions, consolidations, spin-offs, and other market activity.\n\nSo, based on the provided information, Snowflake had 9,437 customers in fiscal year 2024.']